### References

<ul>
    <li>Trending twitter<br></li>
    https://twitter-trends.iamrohit.in/
    <li>Removing accented characters<br></li>
    https://stackoverflow.com/a/2633310/13557629
    <li>Importing package from filepath</li>
    https://stackoverflow.com/a/50395128/13557629
</ul>

In [1]:
import ipyparams
import regex as re
import numpy as np 
import pandas as pd
from importlib import reload
import connection
from datetime import datetime
import utils
import logging

<IPython.core.display.Javascript object>

In [2]:
utils.setup_logger(
    file_name='testing-extraction-naming', 
    desc=''
)

In [11]:
reload(connection)

NameError: name 'twitter_data' is not defined

In [3]:
gen_conf = utils.get_config()
conf = utils.get_config('e')

### Extraction

In [4]:
# TODO: key_name not working properly; python-decouple not finding .env file
#   potentially because it's looking in a different directory
es_conn = connection.TwitterConnection(
    lang='es',
    is_archive=True,
    key_name='SECRET'
)

# pt_conn = connection.TwitterConnection() 

In [5]:
conjugations = pd.read_csv(utils.get_project_root()/'data'/'twitter-conjugations-query.csv')
conjugations = conjugations.to_dict('records')

In [6]:
# List of verbs that shouldn't be pulled
finished = []

In [7]:
time = utils.get_str_datetime_now(True, True)

In [8]:
save_path = utils.make_dir(
    utils.get_save_path('e', 'twitter', False, 'es'), time 
)

for i, topic in enumerate(conjugations):
    # Skip verbs that were already pulled or are not desired
    if topic['verb'] in finished:
        continue
    
    pages = es_conn.paginate(save_path, 
                             (topic['verb'], topic['indicativo']), 
                             batch_size=100,
                             batch_num=1,
                             sleep_sec=1)
    
    print(f'Retrieved total {pages} pages for {topic["verb"]}.')
    
    finished.append(topic['verb'])
    break

Retrieved total 1 pages for ver.
